In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
from sklearn.linear_model import Lasso

X = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')\
.drop(pd.read_csv('../input/math5470/validation_indexes.csv', index_col=0).index).sample(frac=0.5)
X.index = range(X.shape[0])
X.shape

(1178029, 304)

In [2]:

pd.read_csv('../input/math5470/validation_indexes.csv', index_col=0).shape

(785352, 0)

In [3]:
features = X.columns[4:]
corrs = list() 
for col in features:
    corr = np.corrcoef(X['target'], X[col])[0][1]
    corrs.append(corr)
    
corrs_feature_assetNum =pd.Series(corrs, index=features)

In [4]:
import numpy as np
from sklearn.model_selection import KFold
import scipy.stats as st
import xgboost as xgb

feature_subset = corrs_feature_assetNum.sort_values(ascending=False).index

y = X['target']
X = X[feature_subset]

kf = KFold(n_splits=3)
kf.get_n_splits(X)

params_1 = [1e-4,1e-3,1e-2,]
params_2 = [True, False]
scores = {}

for param_1 in params_1:
    for param_2 in params_2:
        scores[(param_1, param_2)] = []
        
for train_index, test_index in kf.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    for param_1 in params_1:
        for param_2 in params_2:
            model = Lasso(alpha=param_1, fit_intercept=param_2)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            scores[(param_1, param_2)] += [st.pearsonr(y_pred, y_test)[0]]
            print(scores[(param_1, param_2)])

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.510e+03, tolerance: 6.640e+01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


[0.1303720662101567]


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.452e+03, tolerance: 6.644e+01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


[0.1298147180489819]
[0.12856085956435978]
[0.12798526438089147]
[0.10545073785243929]
[0.10491149519838462]


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.583e+03, tolerance: 6.650e+01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


...

[0.1298147180489819, 0.1322781404667092]
[0.12856085956435978, 0.1323773952227439]
[0.12798526438089147, 0.13186514453761497]
[0.10545073785243929, 0.11132371528054742]
[0.10491149519838462, 0.11079881730548882]


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.760e+03, tolerance: 6.621e+01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


[0.1303720662101567, 0.1327611762393595, 0.13293992743852184]


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.466e+03, tolerance: 6.625e+01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


[0.1298147180489819, 0.1322781404667092, 0.13239427865653036]
[0.12856085956435978, 0.1323773952227439, 0.13203826836001362]
[0.12798526438089147, 0.13186514453761497, 0.13160380561516463]
[0.10545073785243929, 0.11132371528054742, 0.11033026826267908]
[0.10491149519838462, 0.11079881730548882, 0.10995410785641463]


In [5]:
best_params = pd.DataFrame.from_dict(scores, orient='index').T.mean().dropna().sort_values().index[-1]
best_params

(0.0001, True)

In [6]:
model = Lasso(alpha=best_params[0], fit_intercept=best_params[1])
model.fit(X, y)
model.score(X,y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.721e+03, tolerance: 9.956e+01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


0.018197122336688465

In [7]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['target'] = model.predict(test_df[feature_subset])  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
